## 1. Configuración de conexiones (fuente y bodega)

In [16]:
import yaml
from sqlalchemy import create_engine
import pandas as pd

# Leer configuración desde archivo externo
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# URLs de conexión
url_fuente = (f"{config_fuente['driver']}://{config_fuente['user']}:{config_fuente['password']}@"
              f"{config_fuente['host']}:{config_fuente['port']}/{config_fuente['db']}")

url_bodega = (f"{config_bodega['driver']}://{config_bodega['user']}:{config_bodega['password']}@"
              f"{config_bodega['host']}:{config_bodega['port']}/{config_bodega['db']}")

# Engines SQLAlchemy
engine_fuente = create_engine(url_fuente)
engine_bodega = create_engine(url_bodega)

# 2. Extraccion de datos desde OLTP

In [17]:
df_novedades = pd.read_sql("SELECT * FROM mensajeria_novedadesservicio", engine_fuente)
df_servicio = pd.read_sql("SELECT id AS servicio_id, cliente_id, ciudad_origen_id, ciudad_destino_id, usuario_id, fecha_deseada FROM mensajeria_servicio", engine_fuente)
df_usuario = pd.read_sql("SELECT id AS usuario_id, sede_id FROM clientes_usuarioaquitoy", engine_fuente)
df_estados = pd.read_sql("SELECT servicio_id, estado_id, fecha FROM mensajeria_estadosservicio", engine_fuente)

# 3. Transformaciones

In [18]:
estado_actual = df_estados.sort_values(by='fecha').groupby('servicio_id').tail(1)[['servicio_id', 'estado_id']]
estado_actual.rename(columns={'estado_id': 'id_estado_actual'}, inplace=True)

df = df_novedades.merge(df_servicio, on='servicio_id', how='left')
df = df.merge(df_usuario, on='usuario_id', how='left')
df = df.merge(estado_actual, on='servicio_id', how='left')

df_fecha = pd.read_csv("dim_fecha.csv")
df['fecha_novedad'] = pd.to_datetime(df['fecha_novedad'])
df_fecha['date'] = pd.to_datetime(df_fecha['date']).dt.date
df = df.merge(df_fecha[['id_fecha', 'date']], left_on=df['fecha_novedad'].dt.date, right_on='date', how='left')


df_tiempo = pd.read_csv("dim_tiempo.csv")
df_tiempo['id_tiempo'] = df_tiempo.reset_index().index + 1  # Asegurar columna id_tiempo

df['hora'] = pd.to_datetime(df['fecha_novedad']).dt.hour
df['minuto'] = pd.to_datetime(df['fecha_novedad']).dt.minute

df = df.merge(df_tiempo[['id_tiempo', 'hora', 'minuto']], on=['hora', 'minuto'], how='left')

# Calculo del tiempo de retraso

df['fecha_deseada'] = pd.to_datetime(df['fecha_deseada']).dt.tz_localize(None)
df['fecha_novedad_dt'] = pd.to_datetime(df['fecha_novedad']).dt.tz_localize(None)

df['tiempo_retraso_asociado'] = (df['fecha_novedad_dt'] - df['fecha_deseada']).dt.total_seconds() / 3600


In [19]:
print(df.groupby('id_tiempo').size().sort_values(ascending=False).head())

id_tiempo
988     45
1317    26
728     24
1341    21
824     21
dtype: int64


# 4. Construccion del hecho

In [20]:
hecho_novedades = df[[
    'id', 'id_tiempo', 'id_fecha', 'cliente_id', 'sede_id',
    'ciudad_origen_id', 'ciudad_destino_id', 'mensajero_id',
    'id_estado_actual', 'tipo_novedad_id', 'tiempo_retraso_asociado'
]].rename(columns={
    'id': 'id_novedad',
    'cliente_id': 'id_cliente',
    'sede_id': 'id_sede_cliente',
    'ciudad_origen_id': 'id_ciudad_origen',
    'ciudad_destino_id': 'id_ciudad_destino',
    'mensajero_id': 'id_mensajero',
    'tipo_novedad_id': 'id_tipo_novedad'
})


# 5. Guardar el hecho - Load

In [21]:
# Exportar a CSV
hecho_novedades.to_csv("hecho_novedades.csv", index=False)

# Cargar a base de datos OLAP
hecho_novedades.to_sql("hecho_novedades", engine_bodega, if_exists="replace", index=False)

print("Muestra de la tabla de hechos:")
print(hecho_novedades.head())

print("Filas con id_fecha nulo:", hecho_novedades['id_fecha'].isnull().sum())
print("Filas con id_tiempo nulo:", hecho_novedades['id_tiempo'].isnull().sum())


Muestra de la tabla de hechos:
   id_novedad  id_tiempo  id_fecha  id_cliente  id_sede_cliente  \
0           4        301       183           5                7   
1           5        301       183           5                7   
2           6        301       183           5                7   
3           7        301       183           5                7   
4           8        301       183           5                7   

   id_ciudad_origen  id_ciudad_destino  id_mensajero  id_estado_actual  \
0                 1                  1             7                 5   
1                 1                  1             7                 5   
2                 1                  1             7                 5   
3                 1                  1             7                 5   
4                 1                  1             7                 5   

   id_tipo_novedad  tiempo_retraso_asociado  
0                1                    365.0  
1                1           

In [22]:
hecho_novedades.columns

Index(['id_novedad', 'id_tiempo', 'id_fecha', 'id_cliente', 'id_sede_cliente',
       'id_ciudad_origen', 'id_ciudad_destino', 'id_mensajero',
       'id_estado_actual', 'id_tipo_novedad', 'tiempo_retraso_asociado'],
      dtype='object')